# Financial Engineering - Markov Process Binomial Tree

##### Andrew Cachia, Jan 2018

#### Imports

In [28]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt  
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
from scipy.optimize import minimize
import math

#### Data

In [29]:
date_start = "2013-01-01"
date_end = "2017-12-31"

symbols = ['GOOG']

data = pd.DataFrame()
yf.pdr_override() 
data = pdr.get_data_yahoo(symbols, start=date_start, end=date_end)
data.head()

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-12-31,347.738312,352.989136,345.751221,351.404449,351.404449,4020600
2013-01-02,357.385559,361.151062,355.959839,359.288177,359.288177,5115500
2013-01-03,360.122742,363.600128,358.031342,359.496826,359.496826,4666500
2013-01-04,362.313507,368.339294,361.488861,366.600616,366.600616,5562800
2013-01-07,365.348755,367.301056,362.929504,365.001007,365.001007,3332900


In [30]:
returns = data['Adj Close'] / data['Adj Close'].shift(1)
returns = returns.dropna()
returns = np.log(returns)
returns.head()

Date
2013-01-02    0.022187
2013-01-03    0.000581
2013-01-04    0.019568
2013-01-07   -0.004373
2013-01-08   -0.001975
Name: Adj Close, dtype: float64

### Asset Statistics

In [31]:
yearly_closing = returns.groupby(returns.index.year).sum()
expected_growth = yearly_closing.mean()
std_dev = yearly_closing.std()

simulation_time = 0.5
number_price_changes = 6
deltaT = simulation_time / number_price_changes

up_factor = math.exp(std_dev * math.sqrt(deltaT))
down_factor = 1 / up_factor

up_probability = (math.exp(expected_growth*deltaT)-down_factor) / (up_factor-down_factor)
down_probability = 1 - up_probability

print("Std dev: %5.2f%%" % (std_dev * 100))
print("Expected Growth: %5.2f%%" % (expected_growth * 100))
print("DeltaT: %5.2f%%" % (deltaT * 100))
print("Up Factor: %5.2f%%" % (up_factor * 100))
print("Down Factor: %5.2f%%" % (down_factor * 100))
print("Up probability: %5.2f%%" % (up_probability * 100))
print("Down probability: %5.2f%%" % (down_probability * 100))

Std dev: 22.81%
Expected Growth: 21.82%
DeltaT:  8.33%
Up Factor: 106.81%
Down Factor: 93.63%
Up probability: 62.28%
Down probability: 37.72%


## Project Stock Prices

In [32]:
projected_prices = []
current_price = data['Adj Close'].tail(1).values
#current_price = [100]
projected_prices.append(current_price)



for i in range(1,number_price_changes+1):
    temp = []
    for j in range(0,len(projected_prices[i-1])+1):
        if j==0:
            price = projected_prices[i-1][0] * up_factor
        else:
            price = projected_prices[i-1][j-1] * down_factor
        #print(price)
        temp.append(price)
    #print("--")
    projected_prices.append(temp)
    
tree = pd.DataFrame(projected_prices).T
tree.columns = ['Month'+str(i) for i in range(0,number_price_changes+1)]
tree

,Month0,Month1,Month2,Month3,Month4,Month5,Month6
0,1046.400024,1117.626513,1193.701257,1274.954265,1361.738014,1454.428969,1553.429225
1,NaN,979.712808,1046.400024,1117.626513,1193.701257,1274.954265,1361.738014
2,NaN,NaN,917.275578,979.712808,1046.400024,1117.626513,1193.701257
3,NaN,NaN,NaN,858.817481,917.275578,979.712808,1046.400024
4,NaN,NaN,NaN,NaN,804.084926,858.817481,917.275578
5,NaN,NaN,NaN,NaN,NaN,752.840485,804.084926
6,NaN,NaN,NaN,NaN,NaN,NaN,704.861858


## Project Probabilities

In [33]:
projected_probailities = []
current_prob = [1]
projected_probailities.append(current_prob)

for i in range(1,number_price_changes+1):
    temp = []
    for j in range(0,len(projected_probailities[i-1])+1):
        if j==0:
            prob = projected_probailities[i-1][0] * up_probability
        elif j==len(projected_probailities[i-1]):
            prob = projected_probailities[i-1][j-1] * down_probability
        else:
            prob = projected_probailities[i-1][j-1] * down_probability + projected_probailities[i-1][j] * up_probability
        #print(prob)
        temp.append(prob)
    #print("--")
    projected_probailities.append(temp)
    
tree = pd.DataFrame(projected_probailities).T
tree.columns = ['Month'+str(i) for i in range(0,number_price_changes+1)]
tree

,Month0,Month1,Month2,Month3,Month4,Month5,Month6
0,1.0,0.622791,0.387868,0.241561,0.150442,0.093694,0.058352
1,NaN,0.377209,0.469845,0.438923,0.364476,0.283740,0.212053
2,NaN,NaN,0.142287,0.265845,0.331131,0.343709,0.321088
3,NaN,NaN,NaN,0.053672,0.133706,0.208176,0.259301
4,NaN,NaN,NaN,NaN,0.020246,0.063044,0.117789
5,NaN,NaN,NaN,NaN,NaN,0.007637,0.028537
6,NaN,NaN,NaN,NaN,NaN,NaN,0.002881


### Probability of stock price being greater than starting price in 6 months time

In [34]:
result = projected_probailities[6][0] + projected_probailities[6][1] + projected_probailities[6][2]
print("Probability: %5.2f%%" % (result * 100))

Probability: 59.15%


### Expected stock price on a monthly basis

In [35]:
for i in range(0,number_price_changes+1):
    expected_price = np.dot(projected_prices[i], projected_probailities[i])
    print("Month {}: {}".format(i, expected_price))

Month 0: 1046.400024
Month 1: 1065.6041806826443
Month 2: 1085.1607834905112
Month 3: 1105.0763007248772
Month 4: 1125.3573193971374
Month 5: 1146.0105474074453
Month 6: 1167.0428157633332
